# Introduction: the Proba-3 mission and ASPIICS coronagraph operations

See our website: https://www.sidc.be/proba-3 \
ASPIICS helpdesk: aspiics_support@oma.be

The observations are grouped in campaigns about 2 weeks long, separated by about a month with the instrument off (to compromise on the fuel consumption that is needed to maintain the fine formation flight and that needed to move to and from a "coronagraph" orbit): https://www.sidc.be/proba-3/operation-calendar

During each "coronagraph" orbit, the observations are run 5.5 hours around apogee (hence our more convenient handling by full orbits instead of e.g. full day). \
Details of the observing programs for each orbit: https://www.sidc.be/proba-3/commanding/

The cadence in the different filters can be very irregular from one orbit to another, depending on the science objective but mainly on the available telemetry (due to the limited onboard storage; see same link for more details). Also: some images may be simply missing (forever): beware in case you apply time series analysis methods that need regular time steps (FFT, wavelet, ...). 

More details [in the general presentation](https://cloud-as.oma.be/index.php/s/Q4zqA22R37GtSeq).  

# ASPIICS data

See https://www.sidc.be/proba-3/data \
Six different filters on a filter wheel (so that one cannot get simultaneous images in them): Wideband, He I, Fe XIV, Polarisation 120°, 0°, 60°. 



For each filter, we usually acquire from 1 to 3 different "exposures" with different exposure times, to cope with the large dynamic range in the solar corona (factor 1000 from 1 to 3 Rs). Aka HDR. 

To reduce the data volume (onboard storage and downlink), we use tile maps and "Quality Flag" to discard (most of the) underexposed and saturated pixels. 



### Level-1 
Uncalibrated data (DN). \
Missing pixels have the value of BLANK in the header, i.e. -32768   \
Saturated pixels have the max value in 14 bits, i.e. 16383. 

### Level-2
Calibrated images in Mean Solar Brightness (MSB): dark current, flatfield, non-linearity, ... But:
- no recentering or rotation yet (but the corresponding information is available in CRPIX, CRVAL and CROTA).
- correspond to each individual exposure, with partial FOV (tile maps and Quality Flag)  

Missing pixels have value NaN \
Saturated Pixels have value Inf.

### Level-3
Individual exposures are merged, after recentering to Sun centre, rotation to get Solar North up (corresponding FITS keywords updated accordingly so that your software do not correct again). 

Provided in the following data products: 
- Total solar brightness (F corona removed)
- Polarised brightness
- Emission in the He I line (continuum subtracted)
- Emission in the Fe XIV line (continuum subtracted)

Missing pixels have value NaN \
Saturated Pixels have value Inf.


### FITS file names

Level-1: aspiics_[filter]\_l1_[exposureID]_[utc].fits 

Level-2: aspiics_[filter]\_l2_[exposureID]_[utc].fits

Level-3: aspiics_[datasetID]\_l3_[acquisitionID]_[utc].fits

Where: 
- [filter] is one of "unknown", "wb" (wideband), "p1" (polariser 60°), "p2" (polariser 0°), "p3" (polariser 120°), "fe" (Fe XIV), or "he" (He I).
- [acquisitionID] is an acquisition identifier encoded as 13 hexadecimal characters:
    - [0‐F]{8} = cycle ID
    - [0‐F]{4} = sequence number
    - [1‐6] = acquisition number
- [exposureID] is the [acquisitionID], followed by one of 1, 2, or 3.
- [utc] is a string representation of UTC in the yyyymmddThhmmss format, rounded down to second start
- [datasetID] is a string identifier for the data product: "bt" (total brightness) or "pb" (polarised brightness) (emission in the spectral lines to come in the future)  


### Calibration

The calibration data (matrices of dark current and flatfield, non-linearity, etc) are publicly available. To take into account the degradation of the instrument, a calibration data set applies to a specific time interval in the observation data set. The latest version of the calibration data is available [here](https://gitlab-as.oma.be/P3SC/p3sc_calibration_data_repository/-/releases/permalink/latest). Older versions are available [here](https://gitlab-as.oma.be/P3SC/p3sc_calibration_data_repository/-/releases). Nice documentation not finalised yet, but in the meantime, you can refer to the readme.md. 

The calibration software to produce Level-2 and Level-3 will be made public soon. 


### FITS keywords

More details about the description of FITS keywords can be founs [here](https://cloud-as.oma.be/index.php/s/NTNj85Fa4wKD9S6). 

A few noteworthy keywords (in addition to the classical ones): 
- SCI_OBJ, OBS_MODE, OBS_PRGM
- ORBIT_ID: to arrive soon
- keywords providing information about the 3D geometry of the 2 spacecraft
- position of the exposure into the sequence of exposures
- APS_TEMP: temperature of the detector (e.g. for dark current model)
- commanded and effective tile map (after quality flag action and/or lost tiles), quality flag setting parameters
- PARENT filename(s): Level-1 of the Level-2, and Level-2s of a merged/combined Level-3
- QUALITY: basically, if non-zero, beware of the corresponding anomaly and possible artifacts (e.g. if filter positioning is imperfect), and filter the image if it may mess up with your processing
- coefficients to convert DN or MSB into your favourite Watt, erg or photon units. 

# Data Access 

- public database: 
	- search interface: https://p3sc.oma.be/P3SC_archive/
	- Data directory listing: https://p3sc.oma.be/datarepfiles/

- Download from python: see notebook 2 (courtesy Bram Bourgoignie)


# Data visualisation and analysis

## JHelioviewer

https://www.jhelioviewer.org/

At the moment, ASPIICS images can only be imported manually (from your local drive) as FITS files. Coming soon: imported from JHV interface as JP2000, like images from SDO, SOHO, etc. 

## ASPIICS Map

At the moment: a supplementary library to extend the Sunpy map to ASPIICS data, see notebook 3 (courtesy Bram Bourgoignie)

## ASPIICS color maps

A set of different color maps have been designed to quickly identify the different filters and data products. 
- If you use the ASPIICS map: automatically taken into account. 
- Otherwise, see an example in notebook 4 (courtesy Andreas Debrabandere).

# Calculating photon noise in Level-2 data

(an example of calculating the photon + readout noise in Level-2 data using the Level-1 data and FITS keywords.)